In [ ]:
# First time
%cd /content
!git clone https://github.com/FireMight/point-cloud-retrieval-from-image.git
%cd /content/point-cloud-retrieval-from-image
!git submodule update --init --recursive
%cd ..

In [ ]:
# Pull repository
%cd /content/point-cloud-retrieval-from-image
!git pull --recurse-submodules
%cd ..

In [1]:
import csv
import os
import sys
import math
import numpy as np
from datetime import datetime
from PIL import Image

In [ ]:
# For Google colab
from google.colab import drive
drive.mount('/content/drive')

sdk_dir = '/content/point-cloud-retrieval-from-image/data/oxford/robotcar-dataset-sdk'
project_dir = '/content/drive/My Drive/ADL4CV'

run_id = 'reference'
data_dir = project_dir + '/downloads/oxford_dataset/' + run_id
gps_dir = data_dir + '/gps'
ins_data_file = gps_dir + '/ins.csv'
extrinsics_dir = sdk_dir + '/extrinsics'

In [2]:
# For local use
sdk_dir = 'data/oxford/robotcar-dataset-sdk'

run_id = 'reference'
data_dir = 'data/oxford/data/'  + run_id
gps_dir = data_dir + '/gps'
ins_data_file = gps_dir + '/ins.csv'
extrinsics_dir = sdk_dir + '/extrinsics'

In [3]:
def read_metadata_csv_row(row):
    metadata = {}
    metadata['seg_idx'] = int(row['seg_idx'])
    metadata['timestamp_start'] = int(row['timestamp_start'])
    metadata['northing_start'] = float(row['northing_start'])
    metadata['easting_start'] = float(row['easting_start'])
    metadata['down_start'] = float(row['down_start'])
    metadata['heading_start'] = float(row['heading_start'])
    metadata['timestamp_center'] = int(row['timestamp_center'])
    metadata['northing_center'] = float(row['northing_center'])
    metadata['easting_center'] = float(row['easting_center'])
    metadata['down_center'] = float(row['down_center'])
    metadata['heading_center'] = float(row['heading_center'])
    
    return metadata


def get_pcl_metadata(metadata_file, seg_idx=None):
    if seg_idx is None:
        # Return list of all metadata
        metadata = []
    else:
        # Return metadata of single submap
        metadata = None
        
    with open(metadata_file) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if seg_idx is None:
                metadata.append(read_metadata_csv_row(row))
            elif int(row['seg_idx']) == seg_idx:
                metadata = read_metadata_csv_row(row)
                break
    return metadata

In [4]:
sys.path.insert(0, sdk_dir + '/python')
import image as sdk_image
from camera_model import CameraModel

class ImageLoader():
    def __init__(self, camera_type, center_offset=None):
        self.trajectory_ned = None
        self.subset_split = None
        self.camera_type = camera_type
        self.center_offset = center_offset
        
        if camera_type == 'center':
            self.img_dir = data_dir + '/stereo/centre'
            self.timestamps_file = data_dir + '/stereo.timestamps'
            self.tgt_dir = data_dir + '/img_center'
            if center_offset is not None:
                self.tgt_dir += '_{}'.format(int(center_offset))
        elif camera_type in ['left', 'right']:
            self.img_dir = data_dir + '/mono_{}'.format(camera_type)
            self.timestamps_file = data_dir + '/mono_{}.timestamps'.format(camera_type)
            self.tgt_dir = data_dir + '/img_{}'.format(camera_type)
        else:
            raise ValueError('Wrong camera type: ', camera_type)
        
        if not os.path.isdir(self.tgt_dir):
            os.mkdir(self.tgt_dir)
            
        self.camera_model = CameraModel(sdk_dir + '/models', self.img_dir)
        
    
    def load_ned_trajectory(self, subset_split=None):
        # Get timestamps from camera data
        self.timestamps_file = np.genfromtxt(self.timestamps_file)
        self.timestamps_file = self.timestamps_file[:,0].flatten()
        start_time = self.timestamps_file[0]
        end_time = self.timestamps_file[-1]

        # Check if NED trajectory has already been computed
        trajectory_file = gps_dir + '/ned_trajectory_{}_{}.npy'.format(self.camera_type, 
                                                                       run_id)
        print('Search for precomputed NED trajectory ' + trajectory_file + '...')

        try:
            self.trajectory_ned = np.load(trajectory_file)
            print('Done!')
        except:
            print('Not found! Create camera trajectory...')
            # Get trajectory corresponding to camera data
            camera_timestamp_idx = 0
            self.trajectory_ned = np.empty((7,0))
            with open(ins_data_file, 'r') as ins_file:
                reader = csv.DictReader(ins_file)
                for row in reader:
                    if int(row['timestamp']) > end_time:
                        break
                    if int(row['timestamp']) < start_time:
                        continue

                    # Get closest corresponding camera timestamp
                    ins_timestamp = int(row['timestamp'])
                    camera_timestamp_idx = self.get_closest_camera_timestamp(
                                                ins_timestamp, camera_timestamp_idx)
                    img_timestamp = self.timestamps_file[camera_timestamp_idx]

                    camera_state = np.array([float(row['northing']),
                                             float(row['easting']),
                                             float(row['down']),
                                             float(row['roll']),
                                             float(row['pitch']),
                                             float(row['yaw']),
                                             img_timestamp]).reshape(7,1)
                    self.trajectory_ned = np.append(self.trajectory_ned, camera_state, axis=1)
 
            print('Done! Trajectory with {} samples'.format(
                                                  self.trajectory_ned.shape[1]))

            # Save trajectory
            print('Save NED trajectory to ' + trajectory_file + '...')
            np.save(trajectory_file, self.trajectory_ned)
            print('Done!')
            
        # If specified, use only a subset of the data
        if subset_split is not None:
            max_idx = int(self.trajectory_ned.shape[1] * subset_split)
            print('WARNING: We use only the first {}% of the trajectory ({} measurements)'.
                                                format(int(subset_split*100), max_idx))
            self.trajectory_ned = self.trajectory_ned[:,:max_idx]
            self.subset_split = subset_split
            
    def load_images(self, submap_length):
        # Load pointcloud metadata
        pcl_dir = data_dir + '/submaps_{}m'.format(int(submap_length))
        pcl_metadata_filename = (pcl_dir + '/metadata.csv')
        pcl_metadata = get_pcl_metadata(pcl_metadata_filename)

        # Sort list by timestamp
        pcl_metadata.sort(key=lambda pcl: pcl['timestamp_start'])

        # Create image metadata csv file
        metadata_fieldnames = ['seg_idx',
                               'camera',
                               'offset', 
                               'timestamp',
                               'northing',
                               'easting',
                               'down',
                               'heading', 
                               'dist_to_ref']
        metadata_csv = (self.tgt_dir + '/metadata.csv')

        with open(metadata_csv, 'w') as outcsv:
            writer = csv.DictWriter(outcsv, metadata_fieldnames)
            writer.writeheader()
        
        # Walk along camera trajectory and save all images belonging to a submap
        if self.camera_type == 'center':
            ref = 'start'
            offset = self.center_offset
        else:
            ref = 'center'
            offset = 0
            
        i_pcl = 0
        i_min = 0
        dist_min = 1.0e9
        ref_pos = np.array([pcl_metadata[i_pcl]['northing_'+ref],
                            pcl_metadata[i_pcl]['easting_'+ref]])
                
        for i in range(self.trajectory_ned.shape[1]):
            pos = self.trajectory_ned[:2,i]
            dist_to_ref = np.linalg.norm(pos - ref_pos)

            if dist_to_ref < dist_min:
                i_min = i
                dist_min = dist_to_ref

            if dist_to_ref > dist_min + 0.5:

                # Go back to offset
                if offset > 0:
                    dist_to_offset = offset
                    prev_pos = self.trajectory_ned[:3,i_min]
                    for i_min in range(i_min-1,-1,-1):
                        curr_pos = self.trajectory_ned[[0,1,2],i_min]
                        dist_to_offset -= np.linalg.norm(prev_pos - curr_pos)
                        prev_pos = curr_pos
                        if dist_to_offset < 0:
                            break

                print('Save image {} with dist {}'.format(i_min, dist_min))
                # Save image of step with minimum dist to reference as png
                timestamp = self.trajectory_ned[6,i_min]
                seg_idx = pcl_metadata[i_pcl]['seg_idx']

                _, pil_image = self.load_image(timestamp, size=(320,200))
                pil_image.save(self.tgt_dir + '/img_{}_{}.png'.format(submap_length, 
                                                                         seg_idx), 'PNG')

                # Save image metadata
                with open(metadata_csv, 'a') as outcsv:
                    writer = csv.DictWriter(outcsv,fieldnames=metadata_fieldnames)
                    writer.writerow({'seg_idx' : seg_idx,
                                     'camera' : self.camera_type,
                                     'offset' : self.center_offset, 
                                     'timestamp' : int(timestamp),
                                     'northing' : self.trajectory_ned[0,i_min],
                                     'easting' : self.trajectory_ned[1,i_min],
                                     'down' : self.trajectory_ned[2,i_min],
                                     'heading' : self.trajectory_ned[5,i_min], 
                                     'dist_to_ref' : dist_min})


                # Set new reference position
                i_pcl += 1
                if i_pcl == len(pcl_metadata):
                    print('Last submap reached!')
                    break
                ref_pos = np.array([pcl_metadata[i_pcl]['northing_'+ref],
                                    pcl_metadata[i_pcl]['easting_'+ref]])
                dist_min = 1.0e9
            
        
    
    def get_closest_camera_timestamp(self, ins_timestamp, start_idx):
        min_diff = 1e9
        i_min = start_idx
        for i in range(start_idx, self.timestamps_file.shape[0]):
            diff = math.fabs(ins_timestamp - self.timestamps_file[i])

            if diff < min_diff:
                min_diff = diff
                i_min = i
            else:
                break

        return i_min
    
    def load_image(self, timestamp, size=None):
        image_file = os.path.join(self.img_dir, '{}.png'.format(int(timestamp)))
        image = sdk_image.load_image(image_file, model=self.camera_model)

        pil_image = Image.fromarray(image.astype('uint8'))
        if size is not None:
            pil_image = pil_image.resize(size)

        return image, pil_image

In [5]:
camera_type = 'center'
offset = 5
submap_length = 20

img_loader = ImageLoader(camera_type, center_offset=offset)
img_loader.load_ned_trajectory(subset_split=0.2)
img_loader.load_images(submap_length)

Search for precomputed NED trajectory data/oxford/data/reference/gps/ned_trajectory_center_reference.npy...
Done!
Save image 0 with dist 0.0
Save image 0 with dist 0.008607375699784094
Save image 0 with dist 0.0
Save image 0 with dist 0.0
Save image 805 with dist 0.0
Save image 835 with dist 0.0
Save image 875 with dist 0.0
Save image 895 with dist 0.0
Save image 929 with dist 0.0
Save image 948 with dist 0.0
Save image 972 with dist 0.0
Save image 991 with dist 0.0
Save image 1010 with dist 0.0
Save image 1028 with dist 0.0
Save image 1042 with dist 0.0
Save image 1057 with dist 0.0
Save image 1072 with dist 0.0
Save image 1086 with dist 0.0
Save image 1099 with dist 0.0
Save image 1113 with dist 0.0
Save image 1126 with dist 0.0
Save image 1138 with dist 0.0
Save image 1151 with dist 0.0
Save image 1164 with dist 0.0
Save image 1176 with dist 0.0
Save image 1188 with dist 0.0
Save image 1200 with dist 0.0
Save image 1212 with dist 0.0
Save image 1223 with dist 0.0
Save image 1236 wit

Save image 6086 with dist 0.0
Save image 6094 with dist 0.0
Save image 6101 with dist 0.0
Save image 6108 with dist 0.0
Save image 6114 with dist 0.0
Save image 6121 with dist 0.0
Save image 6127 with dist 0.0
Save image 6133 with dist 0.0
Save image 6139 with dist 0.0
Save image 6145 with dist 0.0
Save image 6152 with dist 0.0
Save image 6158 with dist 0.0
Save image 6164 with dist 0.0
Save image 6170 with dist 0.0
Save image 6176 with dist 0.0
Save image 6182 with dist 0.0
Save image 6189 with dist 0.0
Save image 6195 with dist 0.0
Save image 6201 with dist 0.0
Save image 6207 with dist 0.0
Save image 6213 with dist 0.0
Save image 6219 with dist 0.0
Save image 6225 with dist 0.0
Save image 6231 with dist 0.0
Save image 6237 with dist 0.0
Save image 6243 with dist 0.0
Save image 6249 with dist 0.0
Save image 6255 with dist 0.0
Save image 6261 with dist 0.0
Save image 6267 with dist 0.0
Save image 6273 with dist 0.0
Save image 6279 with dist 0.0
Save image 6285 with dist 0.0
Save image

Save image 8000 with dist 0.0
Save image 8005 with dist 0.0
Save image 8010 with dist 0.0
Save image 8015 with dist 0.0
Save image 8020 with dist 0.0
Save image 8025 with dist 0.0
Save image 8030 with dist 0.0
Save image 8035 with dist 0.0
Save image 8040 with dist 0.0
Save image 8045 with dist 0.0
Save image 8050 with dist 0.0
Save image 8055 with dist 0.0
Save image 8060 with dist 0.0
Save image 8065 with dist 0.0
Save image 8069 with dist 0.0
Save image 8074 with dist 0.0
Save image 8079 with dist 0.0
Save image 8084 with dist 0.0
Save image 8089 with dist 0.0
Save image 8094 with dist 0.0
Save image 8099 with dist 0.0
Save image 8104 with dist 0.0
Save image 8109 with dist 0.0
Save image 8114 with dist 0.0
Save image 8119 with dist 0.0
Save image 8124 with dist 0.0
Save image 8129 with dist 0.0
Save image 8134 with dist 0.0
Save image 8139 with dist 0.0
Save image 8144 with dist 0.0
Save image 8149 with dist 0.0
Save image 8154 with dist 0.0
Save image 8159 with dist 0.0
Save image

Save image 13473 with dist 0.0
Save image 13480 with dist 0.0
Save image 13487 with dist 0.0
Save image 13494 with dist 0.0
Save image 13501 with dist 0.0
Save image 13508 with dist 0.0
Save image 13515 with dist 0.0
Save image 13522 with dist 0.0
Save image 13530 with dist 0.0
Save image 13537 with dist 0.0
Save image 13544 with dist 0.0
Save image 13551 with dist 0.0
Save image 13558 with dist 0.0
Save image 13565 with dist 0.0
Save image 13573 with dist 0.0
Save image 13580 with dist 0.0
Save image 13586 with dist 0.0
Save image 13592 with dist 0.0
Save image 13599 with dist 0.0
Save image 13605 with dist 0.0
Save image 13611 with dist 0.0
Save image 13617 with dist 0.0
Save image 13623 with dist 0.0
Save image 13630 with dist 0.0
Save image 13636 with dist 0.0
Save image 13642 with dist 0.0
Save image 13648 with dist 0.0
Save image 13654 with dist 0.0
Save image 13660 with dist 0.0
Save image 13666 with dist 0.0
Save image 13672 with dist 0.0
Save image 13678 with dist 0.0
Save ima

Save image 15089 with dist 0.0
Save image 15098 with dist 0.0
Save image 15108 with dist 0.0
Save image 15120 with dist 0.0
Save image 15131 with dist 0.0
Save image 15141 with dist 0.0
Save image 15156 with dist 0.0
Save image 15172 with dist 0.0
Save image 15189 with dist 0.0
Save image 15217 with dist 0.0
Save image 15287 with dist 0.0
Save image 15384 with dist 0.0
Save image 15413 with dist 0.0
Save image 15435 with dist 0.0
Save image 15451 with dist 0.0
Save image 15466 with dist 0.0
Save image 15480 with dist 0.0
Save image 15491 with dist 0.0
Save image 15503 with dist 0.0
Save image 15513 with dist 0.0
Save image 15524 with dist 0.0
Save image 15534 with dist 0.0
Save image 15543 with dist 0.0
Save image 15551 with dist 0.0
Save image 15560 with dist 0.0
Save image 15569 with dist 0.0
Save image 15578 with dist 0.0
Save image 15585 with dist 0.0
Save image 15594 with dist 0.0
Save image 15602 with dist 0.0
Save image 15611 with dist 0.0
Save image 15619 with dist 0.0
Save ima

Save image 17557 with dist 0.0
Save image 17563 with dist 0.0
Save image 17569 with dist 0.0
Save image 17576 with dist 0.0
Save image 17582 with dist 0.0
Save image 17588 with dist 0.0
Save image 17594 with dist 0.0
Save image 17600 with dist 0.0
Save image 17607 with dist 0.0
Save image 17613 with dist 0.0
Save image 17619 with dist 0.0
Save image 17625 with dist 0.0
Save image 17631 with dist 0.0
Save image 17637 with dist 0.0
Save image 17643 with dist 0.0
Save image 17649 with dist 0.0
Save image 17655 with dist 0.0
Save image 17660 with dist 0.0
Save image 17667 with dist 0.0
Save image 17673 with dist 0.0
Save image 17679 with dist 0.0
Save image 17684 with dist 0.0
Save image 17690 with dist 0.0
Save image 17696 with dist 0.0
Save image 17702 with dist 0.0
Save image 17708 with dist 0.0
Save image 17714 with dist 0.0
Save image 17720 with dist 0.0
Save image 17726 with dist 0.0
Save image 17732 with dist 0.0
Save image 17737 with dist 0.0
Save image 17743 with dist 0.0
Save ima

Save image 19286 with dist 0.0
Save image 19293 with dist 0.0
Save image 19300 with dist 0.0
Save image 19307 with dist 0.0
Save image 19314 with dist 0.0
Save image 19321 with dist 0.0
Save image 19328 with dist 0.0
Save image 19335 with dist 0.0
Save image 19341 with dist 0.0
Save image 19348 with dist 0.0
Save image 19355 with dist 0.0
Save image 19363 with dist 0.0
Save image 19371 with dist 0.0
Save image 19378 with dist 0.0
Save image 19386 with dist 0.0
Save image 19393 with dist 0.0
Save image 19400 with dist 0.0
Save image 19408 with dist 0.0
Save image 19415 with dist 0.0
Save image 19423 with dist 0.0
Save image 19431 with dist 0.0
Save image 19438 with dist 0.0
Save image 19447 with dist 0.0
Save image 19456 with dist 0.0
Save image 19464 with dist 0.0
Save image 19473 with dist 0.0
Save image 19481 with dist 0.0
Save image 19490 with dist 0.0
Save image 19499 with dist 0.0
Save image 19507 with dist 0.0
Save image 19516 with dist 0.0
Save image 19524 with dist 0.0
Save ima

Save image 22860 with dist 0.0
Save image 22868 with dist 0.0
Save image 22876 with dist 0.0
Save image 22884 with dist 0.0
Save image 22892 with dist 0.0
Save image 22899 with dist 0.0
Save image 22907 with dist 0.0
Save image 22915 with dist 0.0
Save image 22923 with dist 0.0
Save image 22930 with dist 0.0
Save image 22937 with dist 0.0
Save image 22944 with dist 0.0
Save image 22952 with dist 0.0
Save image 22959 with dist 0.0
Save image 22966 with dist 0.0
Save image 22973 with dist 0.0
Save image 22981 with dist 0.0
Save image 22988 with dist 0.0
Save image 22996 with dist 0.0
Save image 23004 with dist 0.0
Save image 23012 with dist 0.0
Save image 23020 with dist 0.0
Save image 23028 with dist 0.0
Save image 23036 with dist 0.0
Save image 23044 with dist 0.0
Save image 23052 with dist 0.0
Save image 23060 with dist 0.0
Save image 23068 with dist 0.0
Save image 23075 with dist 0.0
Save image 23083 with dist 0.0
Save image 23090 with dist 0.0
Save image 23099 with dist 0.0
Save ima

IndexError: list index out of range